<a target="_blank" href="https://colab.research.google.com/github/TinArmEngineering/ltc-docs/blob/main/py-client/monitoring-progress.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
try:
    from google.colab import drive
    #drive.mount('/content/drive')
    IN_COLAB = True
    with open('requirements.txt', 'w') as fout:
        fout.write("""ltc_client>=0.2.3
        pint
        pandas
        pint-pandas
        matplotlib
        pyyaml
        scipy
        openpyxl""")
except:
    IN_COLAB = False


In [ ]:
%%capture --no-display
%pip install -r requirements.txt
%pip install --upgrade jupyter ipywidgets
%jupyter nbextension enable --py widgetsnbextension

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
import matplotlib.pyplot as plt
import ltc_client
import logging
import time
import yaml
import pandas as pd
import pint_pandas
import numpy as np

# Example 2
We assume you have followed Example 1, and have the correct configuration or secrests.


In [ ]:
# are we running in google colab?
if IN_COLAB:
    from google.colab import userdata
    
    print("Running in Google Colab")
    config = {'root_url': "https://api.ltc.tinarmengineering.com",
              'queue_url':"wss://queue.ltc.tinarmengineering.com:15671/ws",
              'api_key': userdata.get("api_key"),
              'org_id': userdata.get("org_id"),
              'queue_user': userdata.get("queue_user"),
              'queue_password': userdata.get("queue_password")}
else:
    
    print("Running locally")
    with open("configurations.yaml", "r") as f:
        config = yaml.safe_load(f)

In [ ]:
LOGGING_LEVEL = logging.INFO
### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)
ch = logging.StreamHandler()
ch.setLevel(LOGGING_LEVEL)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info(f"tinarm version {ltc_client.__version__}")
logger.info(f"pint_pandas version {pint_pandas.__version__}")

q = pint_pandas.PintType.ureg
q.setup_matplotlib()

# Stator
We are using the same machine as before. 

In [ ]:
air_gap_length = 1 * q.mm

stator_parameters = {
    "slot_liner_thickness": 300 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }



## Rotor
Surface mounted Breadloaf magnets

In [ ]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

## Simulation Parameters

In [ ]:
simulation_parameters = {
       "samples_per_electrical_period": 90 * q.count/q.turn,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [ ]:
# calculate the GCD of the number of slots and the number of poles
symmetry = np.gcd(stator_parameters["number_slots"].magnitude, rotor_parameters["number_poles"].magnitude) * q.count
symmetry

winding_parameters = {
    "symmetry": symmetry,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

In [ ]:
materials = {
    "rotor_lamination": "66018e5d1cd3bd0d3453646f",
    "rotor_magnet": "66018e5b1cd3bd0d3453646c",
    "rotor_air_L": "6602fb42c4a87c305481e8a6",
    "rotor_air_R": "6602fb42c4a87c305481e8a6",
    "rotor_banding": "6602fb42c4a87c305481e8a6",
    "stator_lamination": "66018e5d1cd3bd0d3453646f",
    "stator_slot_wedge": "6602fb7239bfdea291a25dd7",
    "stator_slot_liner": "6602fb5166d3c6adaa8ebe8c",
    "stator_slot_winding": "66018e5d1cd3bd0d34536470",
    "stator_slot_potting": "6602fd41b8e866414fe983ec",
}

## Operating Point
This time we are going to evaluate the performace over a grid of simulation parameters. For this case, q axis current densities from 0 to 10 $A/mm^2$, and current angles from 255 degrees to 300

In [ ]:


q_current_density_grid, speed_grid = np.meshgrid(np.linspace(0, 10, 5) * q.A/q.mm**2, np.linspace(15, 4000.0, 5) * q.rpm)
q_current_density_grid.reshape(-1), speed_grid.reshape(-1)

In [ ]:
operating_points = [{
    'q_axis_current_density': j_q,
    'd_axis_current_density': 0 * q.A/q.mm**2,
    'current_angle': 255 * q.degrees,
    "simulated_speed": omega_m,
    } for j_q, omega_m in zip(q_current_density_grid.reshape(-1), speed_grid.reshape(-1))]



In [ ]:
import webstompy
from websocket import create_connection

# Asyncronously monitoring the Jobs.
Make a websocket connection, and connect to the progress STOMP queue

In [ ]:
connection = webstompy.StompConnection(connector=create_connection(config["queue_url"]))
connection.connect(login=config["queue_user"], passcode=config["queue_password"])

In [ ]:
api = ltc_client.Api(config["root_url"], config["api_key"], config["org_id"])

In [ ]:
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]


In [ ]:
#The machine will stay the same in this example, so we can create it once and reuse it.
our_machine = ltc_client.Machine(stator_parameters, rotor_parameters, winding_parameters, materials)
# we actually only do this to get the title.  We could more easily just call the job what we want.
auto_title = "Example2" #ltc_client.Job(our_machine, operating_points[0], simulation_parameters).generate_title()

In [ ]:
'{title}-q{q_axis_current_density:2.4~P}-d{d_axis_current_density:2.4~P}-{simulated_speed:2.4~P}'.format(title=auto_title, **operating_points[0]).replace(" ", "-")

The Job object has a method `to_api()` which returns the json notation of the job that will be posted in the api call [`/jobs/post_jobs`](https://api.ltc.tinarmengineering.com/docs/index.html#/jobs/post_jobs)

In [ ]:
import json

# pretty print the json

job_json = ltc_client.Job(our_machine, operating_points[0], simulation_parameters).to_api()
print(json.dumps(job_json, indent=4))

In [ ]:
jobs = [ltc_client.Job(our_machine, op, simulation_parameters,
        title='{title}-q{q_axis_current_density:2.4~P}-d{d_axis_current_density:2.4~P}-{simulated_speed:2.4~P}'.format(title=auto_title, **op).replace(" ", "-")) for op in operating_points]

from tqdm.auto import trange, tqdm
from tqdm.asyncio import tqdm as asytqdm
from tqdm.contrib.logging import logging_redirect_tqdm

with logging_redirect_tqdm(loggers):
    for job in tqdm(jobs, desc="Creating Jobs"):
        jobs_result = api.create_job(job) 
        tqdm.write(job.id)


In [ ]:
from ltc_client.helpers import async_job_monitor
async def main():
    tasks = [async_job_monitor(api, job, connection=connection, position=idx + 1) for idx,job in enumerate(jobs)] 
    with logging_redirect_tqdm(loggers):
        for f in tqdm.as_completed(tasks, total=len(tasks), unit="Jobs", desc="Jobs", position=0):
            result = await f  # Wait for the task to complete


In [ ]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

asyncio.run(main())

# Summary
In this example we:
1. made a batch of 25 simulations, 
2. made an asyncronus runner and websocket monitor for all 25 jobs,
3. started and monitored all 25 jobs simultaniously.